In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime,date
import re
import json
from pandas.api.types import is_numeric_dtype

# from simple_salesforce import Salesforce, format_soql, format_external_id
# from simple_salesforce.exceptions import SalesforceMalformedRequest
# import simpler_sf
from sqlalchemy import create_engine
#import pymysql

# from mysql import connector


# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [2]:
# usuario = 'siempreread'
# contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "MetLife_A"
plugin_autenticacion = "mysql_native_password"

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

#table.to_sql(name='rut_unico_tiempo',con=motor_mysql_mariadb,if_exists = 'append')


# archivo de salida modificar segun archivo
# archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [3]:
# table.to_sql(name='prueba',con=motor_mysql_mariadb,if_exists = 'append')
rut_unicos = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unicos WHERE LEFT(Rut_T, 1) <> '0' "
rut_unicos = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("-", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("–", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(" ", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.upper()
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.strip()

rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("-", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("–", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(" ", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.upper()
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.strip()

rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(",", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(".", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("-", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┐", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("═", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("╔", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┌", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.title()
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.strip()

rut_unicos.shape



""" rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False
) """

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_7976\3399985333.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_7976\3399985333.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_7976\3399985333.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Nombre"] = rut_unicos["Nombre"].str.r

' rut_unicos.to_excel(\n    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False\n) '

In [4]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unico_tiempo_2 "
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.shape

(6970121, 11)

In [49]:
def sacarultimo(y):
    y = y[:-1]
    return str(y)
    

In [ ]:
sftp = "F:/2024/MetLife/metlife_analisis_todo.xlsx"

derty = pd.read_excel(sftp)

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut"].apply(lambda y: sacarultimo(y))

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].astype(str)
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.upper()
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace("-", "")
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace("–", "")
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].astype(str)
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace(".", "")

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.strip()

derty["rut1"] = derty["rut1"].astype(str)
derty["rut1"] = derty["rut1"].str.upper()
derty["rut1"] = derty["rut1"].str.replace("-", "")
derty["rut1"] = derty["rut1"].str.replace("–", "")
derty["rut1"] = derty["rut1"].astype(str)
derty["rut1"] = derty["rut1"].str.replace(".", "")

derty["rut1"] = derty["rut1"].str.strip()


In [ ]:
rut_unicos2 = pd.merge(
    left=derty,
    right=rut_unico_tiempo_,
    how="left",
    left_on="rut1",
    right_on="rut_rut2",
)


In [ ]:
rut_unicos2.shape

In [ ]:
rut_unicos2.head()

In [ ]:
rut_unicos2 = rut_unicos2.drop_duplicates(
    [
        "rut_rut2","canal_rut","fecha_rut","poliza_rut",
    ],
    keep="last",
)
rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].astype(str)

rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].str.replace("Q", "sftp")
rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].str.replace("correo2", "correo")
rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].str.replace("email", "correo")


In [ ]:
enero2023 = pd.DataFrame()
enero2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023")]
enero2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
enero2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
enero2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

febrero2023 = pd.DataFrame()
febrero2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023")]
febrero2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
febrero2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
febrero2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

marzo2023 = pd.DataFrame()
marzo2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023")]
marzo2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
marzo2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
marzo2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]


abril2023 = pd.DataFrame()
abril2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023")]
abril2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
abril2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
abril2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

mayo2023 = pd.DataFrame()
mayo2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023")]
mayo2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
mayo2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
mayo2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

junio2023 = pd.DataFrame()
junio2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023")]
junio2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
junio2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
junio2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

julio2023 = pd.DataFrame()
julio2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023")]
julio2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
julio2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
julio2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

agosto2023 = pd.DataFrame()
agosto2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023")]
agosto2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
agosto2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
agosto2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

septiembre2023 = pd.DataFrame()
septiembre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023")]
septiembre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
septiembre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
septiembre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

octubre2023 = pd.DataFrame()
octubre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023")]
octubre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
octubre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
octubre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

noviembre2023 = pd.DataFrame()
noviembre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023")]
noviembre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
noviembre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
noviembre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

diciembre2023 = pd.DataFrame()
diciembre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023")]
diciembre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
diciembre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
diciembre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]


enero2024 = pd.DataFrame()
enero2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024")]
enero2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
enero2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
enero2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

febrero2024 = pd.DataFrame()
febrero2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024")]
febrero2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
febrero2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
febrero2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

marzo2024 = pd.DataFrame()
marzo2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024")]
marzo2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
marzo2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
marzo2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

abril2024 = pd.DataFrame()
abril2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024")]
abril2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
abril2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
abril2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

mayo2024 = pd.DataFrame()
mayo2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024")]
mayo2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
mayo2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
mayo2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

#Minera_Abra = Minera_Abra[(Minera_Abra["Empleador"] == Minera_Abra["Nombre Empresa"])]

In [ ]:
enero = pd.DataFrame()
enero = derty.copy()
enero["Enero"]  = enero["rut1"].isin(enero2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero web"]  = enero["rut1"].isin(enero2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero sftp"]  = enero["rut1"].isin(enero2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero correo"]  = enero["rut1"].isin(enero2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Febrero"]  = enero["rut1"].isin(febrero2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero web"]  = enero["rut1"].isin(febrero2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero sftp"]  = enero["rut1"].isin(febrero2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero correo"]  = enero["rut1"].isin(febrero2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Marzo"]  = enero["rut1"].isin(marzo2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo web"]  = enero["rut1"].isin(marzo2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo sftp"]  = enero["rut1"].isin(marzo2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo correo"]  = enero["rut1"].isin(marzo2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Abril"]  = enero["rut1"].isin(abril2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril web"]  = enero["rut1"].isin(abril2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril sftp"]  = enero["rut1"].isin(abril2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril correo"]  = enero["rut1"].isin(abril2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Mayo"]  = enero["rut1"].isin(mayo2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo web"]  = enero["rut1"].isin(mayo2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo sftp"]  = enero["rut1"].isin(mayo2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo correo"]  = enero["rut1"].isin(mayo2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Junio"]  = enero["rut1"].isin(junio2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Junio web"]  = enero["rut1"].isin(junio2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Junio sftp"]  = enero["rut1"].isin(junio2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Junio correo"]  = enero["rut1"].isin(junio2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Julio"]  = enero["rut1"].isin(julio2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Julio web"]  = enero["rut1"].isin(julio2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Julio sftp"]  = enero["rut1"].isin(julio2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Julio correo"]  = enero["rut1"].isin(julio2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Agosto"]  = enero["rut1"].isin(agosto2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Agosto web"]  = enero["rut1"].isin(agosto2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Agosto sftp"]  = enero["rut1"].isin(agosto2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Agosto correo"]  = enero["rut1"].isin(agosto2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Septiembre"]  = enero["rut1"].isin(septiembre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Septiembre web"]  = enero["rut1"].isin(septiembre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Septiembre sftp"]  = enero["rut1"].isin(septiembre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Septiembre correo"]  = enero["rut1"].isin(septiembre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Octubre"]  = enero["rut1"].isin(octubre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Octubre web"]  = enero["rut1"].isin(octubre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Octubre sftp"]  = enero["rut1"].isin(octubre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Octubre correo"]  = enero["rut1"].isin(octubre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Noviembre"]  = enero["rut1"].isin(noviembre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Noviembre web"]  = enero["rut1"].isin(noviembre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Noviembre sftp"]  = enero["rut1"].isin(noviembre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Noviembre correo"]  = enero["rut1"].isin(noviembre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Diciembre"]  = enero["rut1"].isin(diciembre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Diciembre web"]  = enero["rut1"].isin(diciembre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Diciembre sftp"]  = enero["rut1"].isin(diciembre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Diciembre correo"]  = enero["rut1"].isin(diciembre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Enero 2024"]  = enero["rut1"].isin(enero2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero 2024 web"]  = enero["rut1"].isin(enero2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero 2024 sftp"]  = enero["rut1"].isin(enero2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero 2024 correo"]  = enero["rut1"].isin(enero2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Febrero 2024"]  = enero["rut1"].isin(febrero2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero 2024 web"]  = enero["rut1"].isin(febrero2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero 2024 sftp"]  = enero["rut1"].isin(febrero2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero 2024 correo"]  = enero["rut1"].isin(febrero2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Marzo 2024"]  = enero["rut1"].isin(marzo2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo 2024 web"]  = enero["rut1"].isin(marzo2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo 2024 sftp"]  = enero["rut1"].isin(marzo2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo 2024 correo"]  = enero["rut1"].isin(marzo2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Abril 2024"]  = enero["rut1"].isin(abril2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril 2024 web"]  = enero["rut1"].isin(abril2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril 2024 sftp"]  = enero["rut1"].isin(abril2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril 2024 correo"]  = enero["rut1"].isin(abril2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Mayo 2024"]  = enero["rut1"].isin(mayo2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo 2024 web"]  = enero["rut1"].isin(mayo2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo 2024 sftp"]  = enero["rut1"].isin(mayo2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo 2024 correo"]  = enero["rut1"].isin(mayo2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

In [ ]:
enero.head()

In [ ]:
enero.shape

In [ ]:
enero.to_excel("F:/2024/MetLife/datos_metlife_fecha_canal.xlsx", index=False)

In [ ]:
enerob = enero[[
    "rut1","VIGENTE 19.03.2023","EN WEBSERVICE","OBSERVACION","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre","Enero 2024","Febrero 2024","Marzo 2024"
]]

In [ ]:
enerob.head()

In [ ]:
enerob.to_excel("F:/2024/MetLife/datos_metlife_fecha_mes.xlsx", index=False)

In [ ]:
derty["esta"] = derty["rut1"].isin(rut_unicos2["rut_rut2"])
rut_unicos["rut_sin"] = rut_unicos["Rut_T"].apply(lambda y: sacarultimo(y))
derty["unicos"] = derty["rut1"].isin(rut_unicos["rut_sin"])
derty.to_excel("F:/2024/MetLife/datos_metlife_esta.xlsx", index=False)


In [19]:
def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        #print("date_string =", date_string)
        #print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        #print("date_object =", date_object)
        #print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

In [20]:
mes = 5
año = "2024"
fecha = año + "-" + str(mes) + "-" + "01"  
mesesDic = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

str(mesesDic[mes])



sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/sftp.xlsx"


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"


derty = pd.read_excel(sftp)

In [21]:
derty.shape

(454242, 23)

In [15]:
derty.head()

,poliza,RUTASEGURADO,DVASEGURADO,rut2,RUTBENEFICIARIO,DVBENEFICIARIO,Columna2,RUTBENEFIARIO_DV,FNAC,SEXO,...,CIUDAD,TELEFONO,RELACION,desde,hasta,Nombre,TITULAR,EMAIL,CELULAR,VIGENCIA_HASTA
0,340023670,18434052,6,184340526,18434052,6,184340526,6,1993-11-14,M,...,NaN,NaN,AS,2023-01-01,NaN,Riquelme Arias Juan Alejandro,Riquelme Arias Juan Alejandro,NaN,NaN,2024-12-31
1,340022020,18066094,1,180660941,18066094,1,180660941,1,1992-09-08,F,...,SANTIAGO,NaN,AS,2022-08-01,NaN,Zepeda Hernandez Constanza Andrea,Zepeda Hernandez Constanza Andrea,CONSTANZA_ZEPEDA@LIVE.CL,9.799494e+08,2024-10-31
2,340002458,17085671,6,170856716,17085671,6,170856716,6,1989-02-16,M,...,SANTIAGO,NaN,AS,2023-09-01,NaN,Sandoval Salinas Jose Pablo,Sandoval Salinas Jose Pablo,jose.sandoval@latam.com,9.760937e+08,2024-07-31
3,340002973,18124682,0,181246820,22676875,0,226768750,0,2008-03-23,M,...,NaN,09,HI,2023-04-01,NaN,Farias Rodriguez Brandon,Rodriguez Castro Marta Esther,marta.rodriguez@global.komatsu,5.695456e+10,2024-08-31
4,340013428,13357241,4,133572414,13357241,4,133572414,4,1978-11-07,M,...,NaN,NaN,AS,2024-02-01,NaN,Jimenez Chavez Rodrigo,Jimenez Chavez Rodrigo,ro.jimenez711@gmail.com,9.978942e+08,2024-09-30


In [22]:
metlife_sftp = pd.DataFrame()
metlife_sftp = derty.copy()

In [23]:


metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("-", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("–", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(" ", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.upper()
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.strip()

metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("-", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("–", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(" ", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.upper()
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.strip()

metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(",", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(".", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("-", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┐", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("═", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("╔", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┌", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.title()
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.strip()

metlife_sftp["poliza"] = metlife_sftp["poliza"].astype(str)
metlife_sftp["poliza"] = metlife_sftp["poliza"].str.strip()



#*******************sacar primer cero
metlife_sftp.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\3662042864.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\3662042864.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\3662042864.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["Nombre"] = metlife_sftp["Nomb

(454242, 23)

In [24]:
metlife_sftp.head()

,poliza,RUTASEGURADO,DVASEGURADO,rut2,RUTBENEFICIARIO,DVBENEFICIARIO,rut1,RUTBENEFIARIO_DV,Columna1,FNAC,...,CIUDAD,TELEFONO,RELACION,desde,hastad,Nombre,TITULAR,EMAIL,CELULAR,hasta
0,340023670,18434052,6,184340526,18434052,6,184340526,6,1,34287,...,NaN,NaN,AS,44927.0,NaN,Riquelme Arias Juan Alejandro,Riquelme Arias Juan Alejandro,NaN,NaN,45657.0
1,340022020,18066094,1,180660941,18066094,1,180660941,1,1,33855,...,SANTIAGO,NaN,AS,44774.0,NaN,Zepeda Hernandez Constanza Andrea,Zepeda Hernandez Constanza Andrea,CONSTANZA_ZEPEDA@LIVE.CL,979949391.0,45596.0
2,340002458,17085671,6,170856716,17085671,6,170856716,6,1,32555,...,SANTIAGO,NaN,AS,45170.0,NaN,Sandoval Salinas Jose Pablo,Sandoval Salinas Jose Pablo,jose.sandoval@latam.com,976093700.0,45504.0
3,340013428,13357241,4,133572414,13357241,4,133572414,4,1,28801,...,NaN,NaN,AS,45323.0,NaN,Jimenez Chavez Rodrigo,Jimenez Chavez Rodrigo,ro.jimenez711@gmail.com,997894170.0,45565.0
4,340022865,16097460,5,160974605,16097460,5,160974605,5,1,31247,...,NaN,NaN,AS,44896.0,NaN,Espinoza Orostica Maximiliano Ricardo,Espinoza Orostica Maximiliano Ricardo,THE.OROSTICA@GMAIL.COM,965970153.0,45626.0


In [25]:
def siPrimerCero(y):    
    if len(y) <= 4:
        y =  "NONE"
    if y[0] == "0":
        y = y[1:]
    return str(y)
    

In [26]:
metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp = metlife_sftp[(metlife_sftp["rut1"] != "NONE")]
metlife_sftp = metlife_sftp[(metlife_sftp["rut2"] != "NONE")]


metlife_sftp["Titular"] = np.where(metlife_sftp["rut1"] == metlife_sftp["rut2"],1,2 )
metlife_sftp = metlife_sftp[(metlife_sftp["Titular"] == 1)]
metlife_sftp = metlife_sftp.drop_duplicates(
    [
        "rut1","poliza",
    ],
    keep="last",
)

metlife_sftp.shape

(419480, 24)

In [27]:
metlife_sftp.shape

(419480, 24)

In [28]:
correo = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/correo.xlsx"
dertyc = pd.read_excel(correo)
metlife_correo = dertyc.copy()

# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("-", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("–", "")
metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(" ", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.upper()
metlife_correo["rut1"] = metlife_correo["rut1"].str.strip()

metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("-", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("–", "")
metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(" ", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.upper()
metlife_correo["rut2"] = metlife_correo["rut2"].str.strip()

metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(",", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(".", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("-", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┐", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("═", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("╔", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┌", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.title()
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.strip()

metlife_correo["poliza"] = metlife_correo["poliza"].astype(str)
metlife_correo["poliza"] = metlife_correo["poliza"].str.strip()




#*******************sacar primer cero
metlife_correo.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\2005110109.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\2005110109.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\2005110109.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["Nombre"] = metlife

(24265, 8)

In [29]:
metlife_correo["rut1"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo = metlife_correo[(metlife_correo["rut1"] != "NONE")]
metlife_correo = metlife_correo[(metlife_correo["rut2"] != "NONE")]

metlife_correo["Titular"] = np.where(metlife_correo["rut1"] == metlife_correo["rut2"],1,2 )
metlife_correo = metlife_correo[(metlife_correo["Titular"] == 1)]
metlife_correo = metlife_correo.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)

metlife_correo.shape

(18330, 9)

In [ ]:
metlife_correo.shape

In [30]:
web = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/web.xlsx"
dertyd = pd.read_excel(web)


In [31]:
dertyd.head(9)

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,p_paterno,Nombre,poliza,p_relacion,rut2,rut1,p_sexo,procesado,rut_empresa,version
0,539505,NaN,0,NaN,MACHALÍ,77,SIN PLAN,AVENIDA SAN JUAN 202 CASA 12,0,2024-04-01,...,AGUERO,JOSE ROBERTO AGUERO QUINTUL,340023115,1,121185490,121185490,1,0,760503010,1
1,539507,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2024-04-01,...,ZEPEDA,MARCELO OCTAVIO ZEPEDA CORTEZ,340019950,1,126199791,126199791,1,0,967555908,1
2,539508,NaN,0,NaN,RANCAGUA,77,SIN PLAN,ALAMEDA 439 BLOCK F DPTO 21,0,2024-04-01,...,MELLA,EDUARDO MELLA BERDU,340023068,1,126385048,126385048,1,0,969472805,1
3,539511,calama,0,NaN,calama,77,SIN PLAN,El Loa 2465,0,2024-04-01,...,AGUILERA,VERONICA ANDREA AGUILERA COLLAO,340023033,1,157700073,157700073,1,0,767505604,1
4,539515,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2024-04-01,...,PUTNAM,CHRISTIAN PUTNAM VELANDO,340020275,1,219056893,219056893,1,0,651144221,1
5,539521,NaN,0,NaN,ESTACIONCENTRAL,77,SIN PLAN,LAS ACACIAS 65 102 BLOK 9 PISO 1,0,2024-04-01,...,POZO,LAURA CAROL POZO ARAYA,340022932,1,126340435,126340435,2,0,651144221,1
6,539522,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2024-04-01,...,SOTO,JOAQUIN SOTO VARGAS,340002446,1,198921319,198921319,1,0,969512807,1
7,539523,CALAMA,0,NaN,CALAMA,77,SIN PLAN,FRI BONN 2747,0,2024-04-02,...,PERALTA,LUIS PERALTA CUEVAS,340002976,1,157410075,157410075,1,0,651144221,1
8,539526,NaN,0,NaN,MAIPÚ,77,SIN PLAN,JOSE MANUEL BORGOÑO 833 POBLACION,0,2024-04-02,...,CONTRERAS,JAVIER CONTRERAS DIAZ,340019001,1,94670667,94670667,1,0,96557400K,1


In [32]:
metlife_web = dertyd.copy()

In [ ]:
metlife_web.head()
metlife_web.shape

In [33]:


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_web["rut1"] = metlife_web["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut1"] = metlife_web["rut1"].str.replace("-", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace("–", "")
metlife_web["rut1"] = metlife_web["rut1"].astype(str)
metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace(" ", "")
metlife_web["rut1"] = metlife_web["rut1"].str.upper()
metlife_web["rut1"] = metlife_web["rut1"].str.strip()

metlife_web["rut2"] = metlife_web["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].str.replace("-", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace("–", "")
metlife_web["rut2"] = metlife_web["rut2"].astype(str)
metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace(" ", "")
metlife_web["rut2"] = metlife_web["rut2"].str.upper()
metlife_web["rut2"] = metlife_web["rut2"].str.strip()

metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(",", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(".", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("-", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┐", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("═", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("╔", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┌", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.title()
metlife_web["Nombre"] = metlife_web["Nombre"].str.strip()

metlife_web["poliza"] = metlife_web["poliza"].astype(str)
metlife_web["poliza"] = metlife_web["poliza"].str.strip()

metlife_web["rut1"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web = metlife_web[(metlife_web["rut1"] != "NONE")]
metlife_web = metlife_web[(metlife_web["rut2"] != "NONE")]
metlife_web.shape

metlife_web["Titular"] = np.where(metlife_web["rut1"] == metlife_web["rut2"],1,2 )
metlife_web = metlife_web[(metlife_web["Titular"] == 1)]

metlife_web = metlife_web.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_web.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\2932449918.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\2932449918.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\2932449918.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["Nombre"] = metlife_web["Nombre"].

(5350, 41)

In [34]:
webq = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/Q.xlsx"
metlife_Q = pd.read_excel(webq)

metlife_Q["rut1"] = metlife_Q["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace("-", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace("–", "")
metlife_Q["rut1"] = metlife_Q["rut1"].astype(str)
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(".", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(" ", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.upper()
metlife_Q["rut1"] = metlife_Q["rut1"].str.strip()

metlife_Q["rut2"] = metlife_Q["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace("-", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace("–", "")
metlife_Q["rut2"] = metlife_Q["rut2"].astype(str)
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(".", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(" ", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.upper()
metlife_Q["rut2"] = metlife_Q["rut2"].str.strip()

metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace(",", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace(".", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("-", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("┐", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("═", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("╔", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("┌", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.title()
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.strip()

metlife_Q["poliza"] = metlife_Q["poliza"].astype(str)
metlife_Q["poliza"] = metlife_Q["poliza"].str.strip()

metlife_Q["rut1"] = metlife_Q["rut1"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut2"] = metlife_Q["rut2"].apply(lambda y: siPrimerCero(y))
metlife_Q = metlife_Q[(metlife_Q["rut1"] != "NONE")]
metlife_Q = metlife_Q[(metlife_Q["rut2"] != "NONE")]
metlife_Q.shape

metlife_Q["Titular"] = np.where(metlife_Q["rut1"] == metlife_Q["rut2"],1,2 )
metlife_Q = metlife_Q[(metlife_Q["Titular"] == 1)]

metlife_Q = metlife_Q.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_Q.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\3947619704.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\3947619704.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11804\3947619704.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("

(322399, 20)

In [35]:
subir_correo = pd.DataFrame()
subir_correo["poliza_rut"] = metlife_correo["poliza"]
subir_correo["rut_rut"] = metlife_correo["rut1"]
subir_correo["nombre_rut"] = metlife_correo["Nombre"]
subir_correo["mes_rut"] = str(mes)
subir_correo["año_rut"] = str(año)
#subir_correo["fecha_rut"] = metlife_correo["fecha"].apply(lambda y: fechaValida(y))
subir_correo["Llave"] = metlife_correo["poliza"] + metlife_correo["rut1"] + str(mes) + str(año)
subir_correo["desde_rut"] = metlife_correo["desde"].apply(lambda y: fechaValida(y))
subir_correo["hasta_rut"] = metlife_correo["hasta"].apply(lambda y: fechaValida(y))
subir_correo["canal_rut"] = "email"

subir_sql = pd.DataFrame()
subir_sql["poliza_rut"] = metlife_sftp["poliza"]
subir_sql["rut_rut"] = metlife_sftp["rut1"]
subir_sql["nombre_rut"] = metlife_sftp["Nombre"]
subir_sql["mes_rut"] = str(mes)
subir_sql["año_rut"] = str(año)
#subir_sql["fecha_rut"] = metlife_sftp["fecha"].apply(lambda y: fechaValida(y))
subir_sql["Llave"] = metlife_sftp["poliza"] + metlife_sftp["rut1"] + str(mes) + str(año)
subir_sql["desde_rut"] = metlife_sftp["desde"].apply(lambda y: fechaValida(y))
subir_sql["hasta_rut"] = metlife_sftp["hasta"].apply(lambda y: fechaValida(y)) 
subir_sql["canal_rut"] = "sftp"

subir_web = pd.DataFrame()
subir_web["poliza_rut"] = metlife_web["poliza"]
subir_web["rut_rut"] = metlife_web["rut1"]
subir_web["nombre_rut"] = metlife_web["Nombre"]
subir_web["mes_rut"] = str(mes)
subir_web["año_rut"] = str(año)
#subir_web["fecha_rut"] = metlife_web["fecha"].apply(lambda y: fechaValida(y))
subir_web["Llave"] = metlife_web["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_web["desde_rut"] = metlife_web["desde"].apply(lambda y: fechaValida(y))
subir_web["hasta_rut"] = metlife_web["hasta"].apply(lambda y: fechaValida(y))
subir_web["canal_rut"] = "web"

subir_Q = pd.DataFrame()
subir_Q["poliza_rut"] = metlife_Q["poliza"]
subir_Q["rut_rut"] = metlife_Q["rut1"]
subir_Q["nombre_rut"] = metlife_Q["Nombre"]
subir_Q["mes_rut"] = str(mes)
subir_Q["año_rut"] = str(año)
#subir_Q["fecha_rut"] = metlife_Q["fecha"].apply(lambda y: fechaValida(y))
subir_Q["Llave"] = metlife_Q["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_Q["desde_rut"] = metlife_Q["desde"].apply(lambda y: fechaValida(y))
subir_Q["hasta_rut"] = metlife_Q["hasta"].apply(lambda y: fechaValida(y))
subir_Q["canal_rut"] = "correo2"

mes_metlife = pd.DataFrame()
mes_metlife = pd.concat(
    [
        subir_web,
        subir_sql,
        subir_correo,
        subir_Q,
        
    ],
    axis=0,
)
mes_metlife["fecha_rut"] = pd.to_datetime(fecha, yearfirst=True)

mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
mes_metlife.head()

,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,Llave,desde_rut,hasta_rut,canal_rut,fecha_rut
229,340018911,169280029,Kimberly Ninosk Nuñez Sepulveda,5,2024,34001891116928002952024,2024-04-24 13:23:25,2024-04-24 13:23:25,web,2024-05-01
233,340018911,192720737,Ana Valeska Olivares Secan,5,2024,34001891119272073752024,2024-04-24 13:23:25,2024-04-24 13:23:25,web,2024-05-01
234,340018911,89324882,Hugo Antonio Saavedra Araya,5,2024,3400189118932488252024,2024-04-24 13:23:25,2024-04-24 13:23:25,web,2024-05-01
242,340018911,169744572,Diego Ivan Alvarez Castillo,5,2024,34001891116974457252024,2024-04-24 13:23:25,2024-04-24 13:23:25,web,2024-05-01
246,340018911,196852662,Pablo Esteban Diaz Jamett,5,2024,34001891119685266252024,2024-04-24 13:23:25,2024-04-24 13:23:25,web,2024-05-01


//////////////////////////////////////////////////////////////////////////

NUMERO RUTS POR POLIZA
****************************************************************

In [36]:
mes_metlife.shape
print ( str(mes) + "-" + str(año) + " " + str(mes_metlife.shape[0]))

5-2024 429897


////////////////////////////////////////////////

NUMERO RUT UNICOS
------------------------------------------------------------------------------------------------

In [37]:
mes_metlife2 = mes_metlife.drop_duplicates(
    [
        "rut_rut"
    ],
    keep="last",
)
mes_metlife2.shape
print (str(mes) + "-" + str(año) + " " + str(mes_metlife2.shape[0]))

5-2024 330510


In [38]:
#subir_web["Llave"]
mes_metlife["esta"] =  mes_metlife["Llave"].isin(rut_unico_tiempo_["Llave"])
mes_metlife = mes_metlife[(mes_metlife["esta"] == False)]

mes_metlife = mes_metlife[
    [
        "poliza_rut",
        "rut_rut",
        "nombre_rut",
        "mes_rut",
        "año_rut",
        "fecha_rut",
        "Llave",
        "desde_rut",
        "hasta_rut",
        "canal_rut"
    ]
]
mes_metlife.shape

(429897, 10)

In [39]:
mes_metlife.to_sql(
    name="rut_unico_tiempo_2",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

429897

In [40]:
subir_unicos = pd.DataFrame()
subir_unicos["Rut_T"] = mes_metlife2["rut_rut"]
subir_unicos["Rut_C"] = mes_metlife2["rut_rut"]
subir_unicos["Nombre"] = mes_metlife2["nombre_rut"]
subir_unicos["Llave"] = mes_metlife2["rut_rut"] +  mes_metlife2["rut_rut"]


subir_unicos.shape

(330510, 4)

In [41]:
subir_unicos["esta"] =  subir_unicos["Rut_T"].isin(rut_unicos["Rut_T"])
subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]

subir_unicos = subir_unicos[
    [
        "Rut_T",
        "Rut_C",
        "Nombre",
        "Llave",
    ]
]
subir_unicos.shape
subir_unicos.to_sql(
    name="rut_unicos",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

7038

In [45]:
sftp = "F:/2024/MetLife/metlife_analisis_mayo2024.xlsx"
resultado = pd.read_excel(sftp)

In [46]:
resultado.shape

(322401, 19)

In [44]:
rut_unicos.shape

(515659, 5)

In [47]:
rut_unicos["esta"] =  rut_unicos["Rut_T"].isin(resultado["rut2"])

In [10]:
resultado.head()

,poliza,RUTASEGURADO,DVASEGURADO,rut2,RUTBENEFICIARIO,DVBENEFICIARIO,rut1,sdfsdfdsf,Unnamed: 8,RUTBENEFIARIO_DV,...,CIUDAD,TELEFONO,RELACION,desde,hasta,Nombre,TITULAR,EMAIL,CELULAR,VIGENCIA_HASTA
0,340023670,18434052,6,184340526,18434052,6,184340526,1,184340526,6,...,NaN,NaN,AS,2023-01-01,NaT,Riquelme Arias Juan Alejandro,Riquelme Arias Juan Alejandro,NaN,NaN,45657.0
1,340022020,18066094,1,180660941,18066094,1,180660941,1,180660941,1,...,SANTIAGO,NaN,AS,2022-08-01,NaT,Zepeda Hernandez Constanza Andrea,Zepeda Hernandez Constanza Andrea,CONSTANZA_ZEPEDA@LIVE.CL,979949391,45596.0
2,340002458,17085671,6,170856716,17085671,6,170856716,1,170856716,6,...,SANTIAGO,NaN,AS,2023-09-01,NaT,Sandoval Salinas Jose Pablo,Sandoval Salinas Jose Pablo,jose.sandoval@latam.com,976093700,45504.0
3,340022865,16097460,5,160974605,16097460,5,160974605,1,160974605,5,...,NaN,NaN,AS,2022-12-01,NaT,Espinoza Orostica Maximiliano Ricardo,Espinoza Orostica Maximiliano Ricardo,THE.OROSTICA@GMAIL.COM,965970153,45626.0
4,340023194,8563527,1,85635271,8563527,1,85635271,1,85635271,1,...,NaN,56940214308,AS,2023-01-01,NaT,Perez Cifuentes Domingo Luis,Perez Cifuentes Domingo Luis,dperezxtreme@gmail.com,40214308,45657.0


In [48]:
rut_unicos.head()

,Id,Rut_T,Rut_C,Nombre,Llave,esta
0,1,209848961,209848961,Martina Isidora Sanhueza Almendra,209848961209848961,False
1,2,159079465,159079465,Jose Gonzalez Espinoza,159079465159079465,True
2,3,156070432,156070432,Pablo Cesar Aguirre Rodriguez,156070432156070432,False
3,4,153257450,153257450,Daniel Andre Valdes Jimenez,153257450153257450,False
4,5,155558571,155558571,Danilo Enrique Santis Barrera,155558571155558571,False


In [50]:
#subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]
rut_unico_tiempo_11 = rut_unicos[(rut_unicos["esta"]==False)]
#rut_unico_tiempo_11["rutsin"] = rut_unico_tiempo_11["Rut_C"].apply(lambda y: sacarultimo(y))
#rut_unico_tiempo_11 = rut_unico_tiempo_11[(rut_unico_tiempo_11["rutsin"]!=rut_unico_tiempo_11["Rut_C"])]
#rut_unico_tiempo_11["esta2"] =  rut_unico_tiempo_11["rutsin"].isin(rut_unico_tiempo_11["Rut_T"])
rut_unico_tiempo_11.shape

(193260, 6)

In [13]:
rut_unico_tiempo_11 = rut_unicos[(rut_unicos["esta"]==False)]
#rut_unico_tiempo_11["rutsin"] = rut_unico_tiempo_11["Rut_C"].apply(lambda y: sacarultimo(y))
#rut_unico_tiempo_11 = rut_unico_tiempo_11[(rut_unico_tiempo_11["rutsin"]!=rut_unico_tiempo_11["Rut_C"])]
rut_unico_tiempo_11["esta2"] =  rut_unico_tiempo_11["Rut_T"].isin(rut_unico_tiempo_11["rutsin"])
rut_unico_tiempo_11 = rut_unico_tiempo_11[(rut_unico_tiempo_11["esta2"]==False)]

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6844\105263036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rut_unico_tiempo_11["rutsin"] = rut_unico_tiempo_11["Rut_C"].apply(lambda y: sacarultimo(y))


In [32]:
rut_unico_tiempo_12.shape

(9528, 8)

In [31]:
rut_unico_tiempo_12 = rut_unico_tiempo_11[(rut_unico_tiempo_11["esta2"]==True)]

In [21]:
rut_unico_tiempo_11["rutsin"] = rut_unicos["Rut_C"].apply(lambda y: sacarultimo(y))
#rut_unico_tiempo_11 = rut_unico_tiempo_11[(rut_unico_tiempo_11["rutsin"]!=rut_unico_tiempo_11["Rut_C"])]
rut_unico_tiempo_11["esta2"] =  rut_unico_tiempo_11["Rut_T"].isin(rut_unico_tiempo_11["rutsin"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6844\1837957522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rut_unico_tiempo_11["rutsin"] = rut_unicos["Rut_C"].apply(lambda y: sacarultimo(y))
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6844\1837957522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rut_unico_tiempo_11["esta2"] =  rut_unico_tiempo_11["Rut_T"].isin(rut_unico_tiempo_11["rutsin"])


In [ ]:
rut_unico_tiempo_11.head()

In [23]:

resultado.to_excel("F:/2024/MetLife/demases_poliza0.xlsx", index=False)

In [13]:
sftp = "F:/2024/MetLife/demases.xlsx"
resultado = pd.read_excel(sftp)

In [17]:
resultado.head()

,Rut_T,Rut_C,Nombre,Llave,esta,zxczx
0,209848961,209848961,Martina Isidora Sanhueza Almendra,209848961209848961,False,NO
1,156070432,156070432,Pablo Cesar Aguirre Rodriguez,156070432156070432,False,NO
2,153257450,153257450,Daniel Andre Valdes Jimenez,153257450153257450,False,NO
3,155558571,155558571,Danilo Enrique Santis Barrera,155558571155558571,False,NO
4,193277365,193277365,Alejandro Jesus Alvarez Pacheco,193277365193277365,False,NO


In [14]:
rut_unico_tiempo_.head()




,Id,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,fecha_rut,Llave,desde_rut,hasta_rut,canal_rut
0,1,340023027,178055399,Carlos Humberto Araya Jofre,1,2023,2023-01-01,34002302717805539912023,2024-03-25,2024-03-25,web
1,2,340002436,186177908,Nicolas Cisternas Moreno,1,2023,2023-01-01,34000243618617790812023,2024-03-25,2024-03-25,web
2,3,340022737,105796234,Pamela Alejandr Tillemann Munoz,1,2023,2023-01-01,34002273710579623412023,2024-03-25,2024-03-25,web
3,4,340023173,168497261,Pablo Alberto Fredes Fredes,1,2023,2023-01-01,34002317316849726112023,2024-03-25,2024-03-25,web
4,5,340004285,108120045,Rodrigo Alejand Briones Urrutia,1,2023,2023-01-01,34000428510812004512023,2024-03-25,2024-03-25,web


In [16]:
rut_unico_tiempo_2 = rut_unico_tiempo_[
    [
        "rut_rut","poliza_rut"
    ]
]

In [18]:
rut_unico_tiempo_2.head()


,rut_rut,poliza_rut
0,178055399,340023027
1,186177908,340002436
2,105796234,340022737
3,168497261,340023173
4,108120045,340004285


In [19]:
resultado = pd.merge(
    left=resultado,
    right=rut_unico_tiempo_2,
    how="left",
    left_on="Rut_T",
    right_on="rut_rut",
)

In [22]:
resultado.shape

(207192, 8)

In [21]:
#resultado.
resultado = resultado.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)